In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import random
device = "cuda" # the device to load the model onto

/data/xli/miniconda3/envs/xhr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

model = AutoModelForCausalLM.from_pretrained(
    "Qwen2-7B-Instruct",  # path2Qwen2
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen2-7B-Instruct")


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def chat(system, prompt, model=model, tokenizer=tokenizer):

    messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=768
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [6]:
with open('./Eval/dataset/GAOKAO_math.jsonl', 'r') as file:
    content = json.load(file)
content

[{'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知集合 A = {x | -5 < x^3 < 5}, B = {-3, -1, 0, 2, 3}，则 A ∩ B = (  ), 选项是:A: {-1, 0}, B: {2, 3}, C: {-3, -1, 0}, D: {-1, 0, 2}, 您的答案是:',
  'output': 'A'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:若 z / (z - 1) = 1 + i，则 z = (  ), 选项是:A: -1 - i, B: -1 + i, C: 1 - i, D: 1 + i, 您的答案是:',
  'output': 'C'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知向量 ā = (0, 1), b̄ = (2, x)，若 b̄ ⊥ (b̄ - 4ā)，则 x = (  ), 选项是:A: -2, B: -1, C: 1, D: 2, 您的答案是:',
  'output': 'D'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知 cos(α + β) = m, tanαtan β = 2，则 cos(α - β) = (  ), 选项是:A: -3m, B: -m/3, C: m/3, D: 3m, 您的答案是:',
  'output': 'A'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知圆柱和圆锥底面半径相等，侧面半径相等，且它们的高均为 √3，则圆锥的体积为 (  ), 选项是:A: 2√3π, B: 3√3π, C: 6√3π, D: 9√3π, 您的答案是:',
  'output': 'B'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知函数 f(x) = { -x^2 - 2ax - a, x < 0; e^x +

In [9]:
idx = random.randint(0, len(content))
instruction, question, answer = content[idx].values()
prompt = '{question}'.format(question=question)
response = chat(instruction, prompt)
print("The question is:", prompt)
print("The prediction is: ", response)
print("The answer is: ", answer)

The question is: 问题是:已知圆柱和圆锥底面半径相等，侧面半径相等，且它们的高均为 √3，则圆锥的体积为 (  ), 选项是:A: 2√3π, B: 3√3π, C: 6√3π, D: 9√3π, 您的答案是:
The prediction is:  首先，我们知道圆锥的体积公式是 \(V = \frac{1}{3}\pi r^2 h\)，其中 \(r\) 是底面半径，\(h\) 是高。

题目中提到圆锥和圆柱底面半径相等，侧面半径相等，但侧面半径在这个问题中不是直接用到的信息，因为计算体积主要依赖于底面半径 \(r\) 和高 \(h\)。

题目还给出了高 \(h = \sqrt{3}\)。

所以，我们需要找到一个选项符合这个条件。

选项A: \(2\sqrt{3}\pi\)，表示体积为 \(2\sqrt{3}\pi\) 的单位（假设是立方单位）。

选项B: \(3\sqrt{3}\pi\)，表示体积为 \(3\sqrt{3}\pi\) 的单位。

选项C: \(6\sqrt{3}\pi\)，表示体积为 \(6\sqrt{3}\pi\) 的单位。

选项D: \(9\sqrt{3}\pi\)，表示体积为 \(9\sqrt{3}\pi\) 的单位。

由于我们不知道具体的底面半径 \(r\)，但是我们知道，对于任何给定的 \(r\)，如果高 \(h = \sqrt{3}\)，那么体积将会是 \(\frac{1}{3}\pi r^2 \cdot \sqrt{3}\)。

为了匹配给定的选项，我们可以将上述公式简化为：\[V = \frac{\sqrt{3}}{3} \pi r^2\] 这表明，无论 \(r\) 的具体值是多少，体积将总是与 \(\sqrt{3}\) 相关，并且形式上与给出的选项相符。

选项B: \(3\sqrt{3}\pi\)，可以通过将公式 \[V = \frac{\sqrt{3}}{3} \pi r^2\] 与 \(h = \sqrt{3}\) 结合来理解。如果我们将 \(\frac{\sqrt{3}}{3}\) 视为系数，那么要得到 \(3\sqrt{3}\pi\)，实际上是在考虑 \(r^2\) 的部分需要满足某种条件使得整个表达式等于 \(3\sqrt{3}\pi\)。

因此，考虑到给定的选项和题目的描述，正确的答案